# Testing TensorFlow

To install tensorflow I created conda environments: tensorflow (using python 2.7), tensorflow34 (python 3.4) and tensorflow35 (python 3.5) like this

```
conda create -n tensorflow python=2.7
conda create -n tensorflow34 python=3.4
conda create -n tensorflow35 python=3.5
```

After that, I installed tensorflow 0.12 within the tensorflow34 environment, but some functionalities were broken (numpy_input_fn in tf.contrib). So I installed tensorflow 1.0 within the tensor35 environment (because tensorflow 1.0 requires python 3.5). I also installed jupyter within the tensorflow34 environment first, and within the tensorflow35 environment next (`pip install jupyter`).

These are steps to use tensorflow35:

```
source activate tensorflow35
jupyter notebook
```

After using it, deactivate the conda environment:

```
source deactivate
```

Here's the standard test of the tensorflow installation

In [1]:
import tensorflow as tf
import numpy as np

hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

b'Hello, TensorFlow!'


In [2]:
a = tf.constant(10) #implicit tf.float32
b = tf.constant(32)
print(sess.run(a + b))

42


The following code is taken from TensorFlow's getting started page.

## Nodes

In [3]:
node3 = tf.add(a,b)

In [4]:
print(sess.run(node3))

42


## Computing nodes

In [5]:
node_a = tf.placeholder(tf.float32)
node_b = tf.placeholder(tf.float32)

node_adder=node_a + node_b  # + provides a shortcut for tf.add(a, b)

add_and_triple = node_adder * 3.

print(sess.run(node_adder, {node_a: 3, node_b:4.5}))
print(sess.run(node_adder, {node_a: [1,3], node_b: [2, 4]}))
print(sess.run(add_and_triple, {node_a: 3, node_b:4.5}))


7.5
[ 3.  7.]
22.5


## Variables

In [6]:
W = tf.Variable([.3], tf.float32) # Weights
b = tf.Variable([-.3], tf.float32) # intercept (mean values)
x = tf.placeholder(tf.float32) # input vector
linear_model = W * x + b # linear model

Contraty to constants that are self-initialising, variables need to be explicitly initialised

In [7]:
init = tf.global_variables_initializer()
sess.run(init)

In [8]:
print(sess.run(linear_model, {x:[1,2,3,4]}))


[ 0.          0.30000001  0.60000002  0.90000004]


Now we add a placeholder y for data, a squared difference function, and a loss function, which is the sum of the squared differences

In [9]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]})) # (if y were [0,0.3,0.6,0.9] the result would be 0)

23.66


If we manually assign the optimal values for W and b, we can get a model that perfectly fits the data (the single value data...). Now, the loss will be 0:

In [10]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x:[1,2,3,4], y:[0,-1,-2,-3]}))

0.0


## Train

This is a toy example of automatically finding the correct values of W and b by gradient descent

In [11]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x:[1,2,3,4], y:[0,-1,-2,-3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


## The layout of a complete model trainning:

In [12]:
# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1,2,3,4]
y_train = [0,-1,-2,-3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x:x_train, y:y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


## Contrib: packed models

tf.contrib.learn provides a series of wrappers for

* running training loops
* running evaluation loops
* managing datasets
* managing feeding

Here's the same linear model implemented using contrib:

In [13]:
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]
estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)
x = np.array([1., 2., 3., 4.])
y = np.array([0., -1., -2., -3.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x":x}, y, batch_size=4, num_epochs=1000)
estimator.fit(input_fn=input_fn, steps=1000)
estimator.evaluate(input_fn=input_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_task_id': 0, '_tf_random_seed': None, '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_evaluation_master': '', '_num_ps_replicas': 0, '_task_type': None, '_environment': 'local', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11373b400>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_max': 5, '_is_chief': True, '_save_checkpoints_steps': None, '_save_summary_steps': 100}
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/w1/qnhkrsc

{'global_step': 1000, 'loss': 2.2530055e-06}